In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('data/measurements_sleep_0.csv')

start_time = df.iloc[-2]
end_time = df.iloc[-1]

print(start_time)
print(end_time)

# Omit the last two rows
df = df.iloc[:-2]

# Now df contains the DataFrame without the last two rows
print(df)
